In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 81kB 2.0MB/s 
     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 1.2MB 39.5MB/s 
     |████████████████████████████████| 3.3MB 50.3MB/s 
     |████████████████████████████████| 901kB 49.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=888ccb8048137589e6993fabb2dbe027e85c53fa31090c820cf5fa0a7d95e898
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
!wget https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0 -O grouped_data.pickle

--2021-05-04 19:27:49--  https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xq4vosn9xyn1dy1/grouped_data.pickle [following]
--2021-05-04 19:27:49--  https://www.dropbox.com/s/raw/xq4vosn9xyn1dy1/grouped_data.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucad8d39f0aa58e4d1caec60e4bc.dl.dropboxusercontent.com/cd/0/inline/BN2vFz8rpfuf1I9-rpEVnDMKQ6AGAcHq57XuC23nXDBLtlXhBl0xeSHXIG4DBnqfhzHzfc5uq1Nmnih6ytGWqUP1jwDNupshAikoapPR2yPFzswDaejkpJb0zjHNpTeTUI0cw_CCfzuEnv9FcVDYLfp5/file# [following]
--2021-05-04 19:27:50--  https://ucad8d39f0aa58e4d1caec60e4bc.dl.dropboxusercontent.com/cd/0/inline/BN2vFz8rpfuf1I9-rpEVnDMKQ6AGAcHq57XuC23nXDBLtlXhBl0xeSHXIG4DBnqf

In [4]:
!nvidia-smi

Tue May  4 19:28:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import pickle

import torch
!pip install transformers
from transformers import BertForNextSentencePrediction, BertTokenizerFast, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.notebook import tqdm
from scipy import stats
import sklearn
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
!pip install transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
import numpy as np
import torch.optim as optim
import pandas as pd
import torch
import copy
import pickle
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm


from google.colab import drive

random.seed(2021)
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print("You are using device: %s" % device)

You are using device: cuda


In [48]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target):
    # """Computes the precision@k for the specified values of k"""
    # batch_size = target.shape[0]
    # _, pred = torch.max(output, dim=-1)
    # correct = pred.eq(target).sum() * 1.0
    # acc = correct / batch_size
    # return acc
    output = (output > 0.5) 
    return sklearn.metrics.accuracy_score(target.cpu(), output.cpu())

def train(model, dataloader, optimizer, criterion, scheduler = None):
    losses = AverageMeter()
    acc = AverageMeter()

    model.train()

    # Get the progress bar for later modification
    # progress_bar = tqdm(dataloader)

    # Mini-batch training
    # for batch_idx, (data, target) in enumerate(progress_bar):
    for batch_idx, (data, target) in enumerate(dataloader):
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        out = model(data)
        # print(data.shape, out.shape, target.shape)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()

        batch_acc = accuracy(out, target)

        losses.update(loss, out.shape[0])
        acc.update(batch_acc, out.shape[0])
        
    
    return acc.avg, losses.avg

def evaluate(model, dataloader, criterion):

    losses = AverageMeter()
    acc = AverageMeter()
    
    # Set the model to eval mode to avoid weights update
    model.eval()

    with torch.no_grad():
        # Get the progress bar 
        # progress_bar = tqdm(dataloader)
        # for batch_idx, (data, target) in enumerate(progress_bar):
        for batch_idx, (data, target) in enumerate(dataloader):
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()
        
            
#         progress_bar = tqdm(dataloader)
#         for (data, target) in enumerate(dataloader):          

            out = model(data)

            batch_acc = accuracy(out, target)
            loss = criterion(out, target)
            
            losses.update(loss, out.shape[0])
            acc.update(batch_acc, out.shape[0])
            
#             progress_bar.set_description_str("Batch: %d, Loss: %.4f" % ((batch_idx+1), loss.val))
    
    return acc.avg, losses.avg

In [14]:
# load sample data
# format { post_id: [post, [comment1, comment2, ... ] }

data_file_path = 'grouped_data.pickle'

def get_best_comm(comms):
  best_comm = None
  best_score = -np.inf

  for comm in comms:
    if comm['score'] > best_score:
      best_comm = comm
      best_score = comm['score']

    return best_comm

def load_data(file_path):
    posts_comments = []
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        for post_id, val in tqdm(list(data.items())[:5000]):
            post = val[0]
            comms = val[1]
            # best_comm = get_best_comm(comms)

            # if len(post['title']) < 500 and len(best_comm['body']) < 500 and best_comm['body'] != '[deleted]' and best_comm['body'] != '[removed]':
            #   posts_comments.append((post['title'], best_comm['body']))

            for com in comms:
                try:
                    if len(post['title']) < 500 and len(com['body']) < 500 and com['body'] != '[deleted]' and com['body'] != '[removed]':
                        posts_comments.append((post['title'], com['body']))
                except:
                    pass
                
    return posts_comments
    
posts_comms = load_data(data_file_path)
len(posts_comms)

65242

In [19]:
post_comms_realfake = []
labels = []
for post, comment in posts_comms:
  while True:
    rpost, rcomment = random.choice(posts_comms)
    if post != rpost:
      break
  
  post_comms_realfake.append((post, comment))
  labels.append(0) # comment \in post
  post_comms_realfake.append((post, rcomment))
  labels.append(1) # comment \nin post

len(post_comms_realfake)

130484

In [20]:
# split text
train_texts, test_texts, train_labels, test_labels = train_test_split(post_comms_realfake, labels, test_size=.25, shuffle=True)
# train_texts, test_texts, train_labels, test_labels = train_test_split(posts_comms, upvotes, test_size=.1, train_size=.2, shuffle=True)
print(len(train_texts), len(train_labels), len(test_texts), len(test_labels))

posts_comms = None

97863 97863 32621 32621


In [21]:
model = SentenceTransformer('stsb-distilroberta-base-v2')

In [22]:
train_posts = [item[0] + " " + item[1] for item in train_texts]
test_posts = [item[0] + " " + item[1] for item in test_texts]

In [23]:
embed_train = model.encode(train_posts)

In [24]:
embed_test = model.encode(test_posts)

In [25]:
class RelevancyClassifier(nn.Module):
    def __init__(self):
        super(RelevancyClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features=768, out_features=1000, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1000, out_features=500, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=500, out_features=200, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=200, out_features=2, bias=True),
            nn.Softmax()
        )

    def forward(self, x):
        # for layer in self.model:
        #   x = layer(x)
        #   print(layer, x.shape)
        # outs = x

        outs = self.model(x)
        return outs

In [49]:
model = RelevancyClassifier().to(device)

In [50]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.1)
batch_size = 64

In [51]:
class upvote_prediction_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = torch.tensor(self.encodings[idx]).flatten()
        label = torch.tensor(self.labels[idx]).float().flatten()
        return (item, label)

    def __len__(self):
        return len(self.labels)

train_set = upvote_prediction_dataset(embed_train, F.one_hot(torch.as_tensor(train_labels), num_classes=2))
valid_set = upvote_prediction_dataset(embed_test, F.one_hot(torch.as_tensor(test_labels), num_classes=2))


train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False)

In [52]:
losses = []
accs = []

epochs = 10

best_cm = None
best_model = None
best = -np.inf

model.train()
progress_bar = tqdm(range(epochs))
for epoch in progress_bar:

    train_acc, train_loss = train(model, train_loader, optimizer, criterion)

    val_acc, val_loss = evaluate(model, valid_loader, criterion)

    accs.append(val_acc)
    losses.append(val_loss)

    progress_bar.set_description_str("Epoch: %d, Loss: %.4f" % ((epoch+1), val_loss))

    # if acc > best:
        # best = acc
        # best_cm = cm
        # best_model = copy.deepcopy(model)
        # torch.save(best_model.statedict(), './checkpoints/' + args.model.lower() + '' + args.cfg + '.pth')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [19]:
from sklearn import svm

In [20]:
from sklearn.decomposition import PCA# Make an instance of the Model
pca = PCA(n_components=10)

In [21]:
PCA_train = pca.fit_transform(embed_train)
PCA_test = pca.fit_transform(embed_test)

In [22]:
clf = svm.SVC(verbose=True)

In [ ]:
clf.fit(PCA_train, train_labels)

[LibSVM]

In [67]:
model.eval()
y_pred = model(torch.as_tensor(embed_test).to(device))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [68]:
y_pred = y_pred.cpu().detach().numpy()
y_pred = (y_pred > 0.5) 

In [74]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(F.one_hot(torch.as_tensor(test_labels), num_classes=2), y_pred))
print("Precision:",metrics.precision_score(F.one_hot(torch.as_tensor(test_labels), num_classes=2), y_pred), average='None')
print("Recall:",metrics.recall_score(F.one_hot(torch.as_tensor(test_labels), num_classes=2), y_pred))
print("f1-score:",metrics.f1_score(F.one_hot(torch.as_tensor(test_labels), num_classes=2), y_pred))

Accuracy: 0.5005058091413507


ValueError: ignored